In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pyquaternion import Quaternion
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

In [2]:
# Load the Excel file
df_ref = pd.read_excel('Filtered_PCD_Annotations_AB.xlsx')

In [3]:
# Get the object perspective 
def get_object_perspective(cam_pose_x, cam_pose_y, box_x, box_y, box_z, box_rotation_w, box_rotation_z):
    """
    Determines the perspective of an object relative to the camera.

    Parameters:
    - box_center: np.array([x, y, z]) → center of the box in world coordinates
    - orientation_quat: Quaternion(w, 0, 0, z) → rotation around Z-axis only
    - cam_position: np.array([x, y, z]) → camera position in world coordinates
    - threshold: angle threshold in radians for classification

    Returns:
    - One of: 'toward', 'away', 'left', 'right'
    """
    threshold=np.pi / 4

    # Camera position in global coordinates
    cam_position = np.array([cam_pose_x, cam_pose_y, 0])  # shape: (3,)

    orientation_quat = Quaternion(
        box_rotation_w,
        0.0, 0.0,
        box_rotation_z
    )
    
    # Compute forward direction of the box (local +X axis rotated by orientation)
    forward_vector = orientation_quat.rotate(np.array([1, 0, 0]))  # shape: (3,)

    # Vector from box to camera
    box_center = [box_x, box_y, box_z]
    to_camera = cam_position - box_center
    to_camera = to_camera / np.linalg.norm(to_camera)

    # Compute angle between forward direction and camera vector
    dot = np.dot(forward_vector[:2], to_camera[:2])  # only XY plane
    angle = np.arccos(np.clip(dot, -1.0, 1.0))  # radians

    # Pad vectors to 3D by adding a zero Z-component
    forward_xy = np.array([forward_vector[0], forward_vector[1], 0])
    to_camera_xy = np.array([to_camera[0], to_camera[1], 0])
    
    # Compute cross product and extract Z-component
    cross_z = np.cross(forward_xy, to_camera_xy)[2]


    # Classify based on angle and cross product
    if angle < threshold:
        return 0 #'toward'
    elif angle > (np.pi - threshold):
        return 1 #'away'
    elif cross_z > 0:
        return 2 #'left'
    else:
        return 3 #'right'

In [4]:
# Load the Excel file

df = df_ref.copy()

selected_columns = [
    'a_camera_pose_x', 'a_camera_pose_y', 'a_camera_pose_z', 
    'b_camera_pose_x', 'b_camera_pose_y', 'b_camera_pose_z', 
    'a_center_x', 'a_center_y', 'a_center_z', 
    'b_center_x', 'b_center_y', 'b_center_z', 
    'a_length', 'a_width', 'a_height', 
    'b_length', 'b_width', 'b_height', 
    'a_rotation_w', 'a_rotation_x', 'a_rotation_y', 'a_rotation_z', 
    'b_rotation_w', 'b_rotation_x', 'b_rotation_y', 'b_rotation_z', 
    'a_yaw', 
    'b_yaw',
    'a_category',
    'b_category', 
    'a_brake', 'a_brake_switch', 'a_rear_left_rpm', 'a_rear_right_rpm', 'a_vehicle_speed', 'a_yaw_rate',
    'b_brake', 'b_brake_switch', 'b_rear_left_rpm', 'b_rear_right_rpm', 'b_vehicle_speed', 'b_yaw_rate',
    'a_distance', 
    'b_distance', 
    'a_vx', 'a_vy',             
    'b_vx', 'b_vy']
    


df = df[selected_columns]


# Encode box1_category_name
le_category = LabelEncoder()
df['a_category_encoded'] = le_category.fit_transform(df['a_category'])


df['a_box_perspective'] = df.apply(
    lambda row: get_object_perspective(
        row['a_camera_pose_x'],
        row['a_camera_pose_y'],
        row['a_center_x'],
        row['a_center_y'],
        row['a_center_z'],
        row['a_rotation_w'],
        row['a_rotation_z']
    ),
    axis=1
)

df_cleaned_data = df.copy()

df_cleaned_data = df_cleaned_data.drop_duplicates()

# Translation deltas
df_cleaned_data['delta_center_x'] = df_cleaned_data['b_center_x'] - df_cleaned_data['a_center_x']
df_cleaned_data['delta_center_y'] = df_cleaned_data['b_center_y'] - df_cleaned_data['a_center_y']
df_cleaned_data['delta_center_z'] = df_cleaned_data['b_center_z'] - df_cleaned_data['a_center_z']

df_cleaned_data['delta_length'] = df_cleaned_data['b_length'] - df_cleaned_data['a_length']
df_cleaned_data['delta_width'] = df_cleaned_data['b_width'] - df_cleaned_data['a_width']
df_cleaned_data['delta_height'] = df_cleaned_data['b_height'] - df_cleaned_data['a_height']


df_cleaned_data['delta_vehicle_speed'] = df_cleaned_data['b_vehicle_speed'] - df_cleaned_data['a_vehicle_speed']

def compute_delta_quaternion(row):
    q1 = Quaternion([row['a_rotation_w'], 0.0, 0.0, row['a_rotation_z']])
    q2 = Quaternion([row['b_rotation_w'], 0.0, 0.0, row['b_rotation_z']])
    delta_q = q2 * q1.inverse
    return pd.Series({'delta_rotation_w': delta_q.w, 'delta_rotation_x': delta_q.x, 'delta_rotation_y': delta_q.y,'delta_rotation_z': delta_q.z})

df_quat_delta = df_cleaned_data.apply(compute_delta_quaternion, axis=1)
df_cleaned_data = pd.concat([df_cleaned_data, df_quat_delta], axis=1)

# Remove columns with only one unique value
df_cleaned_data = df_cleaned_data.loc[:, df_cleaned_data.nunique() > 1]


df_cleaned_data['a_category_encoded'] = df_cleaned_data['a_category_encoded'].astype('category')

df_cleaned_data = df_cleaned_data.drop(['a_category', 'b_category'], axis=1)

# Select numeric features
non_numeric_cols = df_cleaned_data.select_dtypes(exclude='number').columns.tolist()
df_cleaned_data = pd.get_dummies(df_cleaned_data, columns=non_numeric_cols, drop_first=True)

df_cleaned_data.shape

(5376, 62)

In [6]:

# Select numeric features
numeric_df = df_cleaned_data.select_dtypes(include='number')

numeric_df = numeric_df.drop(['a_vx', 'a_vy', 'b_vx', 'b_vy'], axis=1)  

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(numeric_df)

pca = PCA(n_components=min(10, X_scaled.shape[1]))  # You can adjust n_components
X_pca = pca.fit_transform(X_scaled)
y1 = df_cleaned_data['a_vx']
y2 = df_cleaned_data['a_vy']


X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(
    X_pca, y1, y2, test_size=0.2, random_state=43
)


In [7]:
numeric_df.columns

Index(['a_camera_pose_x', 'a_camera_pose_y', 'b_camera_pose_x',
       'b_camera_pose_y', 'a_center_x', 'a_center_y', 'a_center_z',
       'b_center_x', 'b_center_y', 'b_center_z', 'a_length', 'a_width',
       'a_height', 'b_length', 'b_width', 'b_height', 'a_rotation_w',
       'a_rotation_z', 'b_rotation_w', 'b_rotation_z', 'a_yaw', 'b_yaw',
       'a_brake', 'a_brake_switch', 'a_rear_left_rpm', 'a_rear_right_rpm',
       'a_vehicle_speed', 'a_yaw_rate', 'b_brake', 'b_brake_switch',
       'b_rear_left_rpm', 'b_rear_right_rpm', 'b_vehicle_speed', 'b_yaw_rate',
       'a_distance', 'b_distance', 'a_box_perspective', 'delta_center_x',
       'delta_center_y', 'delta_center_z', 'delta_vehicle_speed',
       'delta_rotation_w', 'delta_rotation_z'],
      dtype='object')

In [8]:


models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'SVR': SVR()
}

In [9]:

results = {}

for name, model in models.items():
    model.fit(X_train, y1_train)
    y1_pred = model.predict(X_test)
    results[name] = {
        'RMSE': np.sqrt(mean_squared_error(y1_test, y1_pred)),
        'MAE': mean_absolute_error(y1_test, y1_pred),
        'R2': r2_score(y1_test, y1_pred)
    }

In [10]:


nn_model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(8, activation='relu'),
    Dense(1)
])

nn_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mae'])

nn_model.fit(X_train, y1_train, epochs=100, batch_size=32, verbose=0)

y1_pred_nn = nn_model.predict(X_test).flatten()
results['NeuralNet'] = {
    'RMSE': np.sqrt(mean_squared_error(y1_test, y1_pred_nn)),
    'MAE': mean_absolute_error(y1_test, y1_pred_nn),
    'R2': r2_score(y1_test, y1_pred_nn)
}

c:\Users\dilip\anaconda3\envs\radarviz\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [11]:

results_df = pd.DataFrame(results).T
print(results_df)

                      RMSE       MAE        R2
LinearRegression  2.903320  1.846974  0.457143
RandomForest      1.140691  0.493589  0.916202
GradientBoosting  1.778115  1.036279  0.796382
SVR               1.843021  0.876464  0.781246
NeuralNet         1.351599  0.809062  0.882350


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance

# Step 1: Hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y1_train)

best_rf = grid_search.best_estimator_

# Step 2: Predict and evaluate
y1_pred_rf = best_rf.predict(X_test)
results['RandomForest_Tuned'] = {
    'RMSE': np.sqrt(mean_squared_error(y1_test, y1_pred_rf)),
    'MAE': mean_absolute_error(y1_test, y1_pred_rf),
    'R2': r2_score(y1_test, y1_pred_rf)
}

# Print results
print("🔍 Random Forest Performance:")
for metric, value in results['RandomForest_Tuned'].items():
    print(f"{metric}: {value:.4f}")

# Step 3: Residual analysis
#residuals = y1_test - y1_pred_rf
#plt.figure(figsize=(6, 4))
#plt.hist(residuals, bins=50, color='teal', alpha=0.7)
#plt.title("Residuals Distribution (Random Forest)")
#plt.xlabel("Error")
#plt.ylabel("Frequency")
#plt.tight_layout()
#plt.show()

🔍 Random Forest Performance:
RMSE: 1.1057
MAE: 0.4852
R2: 0.9213


In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'SVR': SVR()
}

In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

results = {}

for name, model in models.items():
    model.fit(X_train, y2_train)
    y2_pred = model.predict(X_test)
    results[name] = {
        'RMSE': np.sqrt(mean_squared_error(y2_test, y2_pred)),
        'MAE': mean_absolute_error(y2_test, y2_pred),
        'R2': r2_score(y2_test, y2_pred)
    }

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

nn_model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(8, activation='relu'),
    Dense(1)
])

nn_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mae'])

nn_model.fit(X_train, y2_train, epochs=100, batch_size=32, verbose=0)

y2_pred_nn = nn_model.predict(X_test).flatten()
results['NeuralNet'] = {
    'RMSE': np.sqrt(mean_squared_error(y2_test, y2_pred_nn)),
    'MAE': mean_absolute_error(y2_test, y2_pred_nn),
    'R2': r2_score(y2_test, y2_pred_nn)
}

c:\Users\dilip\anaconda3\envs\radarviz\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [16]:
import pandas as pd
import matplotlib.pyplot as plt

results_df = pd.DataFrame(results).T
print(results_df)


                      RMSE       MAE        R2
LinearRegression  1.268204  0.913297  0.733753
RandomForest      0.380021  0.157075  0.976093
GradientBoosting  0.620783  0.365812  0.936205
SVR               0.732179  0.345551  0.911255
NeuralNet         0.511090  0.283217  0.956758


In [19]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance

# Step 1: Hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y2_train)

best_rf = grid_search.best_estimator_

# Step 2: Predict and evaluate
y2_pred_rf = best_rf.predict(X_test)
results['RandomForest_Tuned'] = {
    'RMSE': np.sqrt(mean_squared_error(y2_test, y2_pred_rf)),
    'MAE': mean_absolute_error(y2_test, y2_pred_rf),
    'R2': r2_score(y2_test, y2_pred_rf)
}

# Print results
print("🔍 Random Forest Performance:")
for metric, value in results['RandomForest_Tuned'].items():
    print(f"{metric}: {value:.4f}")

# Step 3: Residual analysis
#residuals = y1_test - y1_pred_rf
#plt.figure(figsize=(6, 4))
#plt.hist(residuals, bins=50, color='teal', alpha=0.7)
#plt.title("Residuals Distribution (Random Forest)")
#plt.xlabel("Error")
#plt.ylabel("Frequency")
#plt.tight_layout()
#plt.show()

🔍 Random Forest Performance:
RMSE: 0.3704
MAE: 0.1557
R2: 0.9773
